# Bin Packing

Bin packing is the problem of efficiently arranging a set of smaller rectangles (items) within a larger rectangle (bin) to minimize wasted space. It has applications in manufacturing, logistics, and computer science.

![bin_packing](./MIT-Spectral-Packing.png)

## Relevance in Architecture and Fabrication

*   **Digital Fabrication (Nesting):** The most direct application. Arranging parts on a standard sheet of plywood, metal, or acrylic for CNC/Laser cutting to minimize material waste and cost.
*   **Floor Plan Generation:** In automated layout design, rooms can be treated as rectangles with required areas. Bin packing algorithms can generate initial "block diagrams" or adjacency layouts.
*   **Texture Atlases:** In computer graphics and rendering, packing many small textures into one large image file to optimize memory usage.
*   **Parking Lot Layouts:** Efficiently arranging parking spots (rectangles) within a site boundary.

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# 1. Generate Random Rectangles
def generate_rectangles(n=20, min_w=5, max_w=20, min_h=5, max_h=20):
    rects = []
    for i in range(n):
        w = random.randint(min_w, max_w)
        h = random.randint(min_h, max_h)
        rects.append({'id': i, 'w': w, 'h': h})
    return rects

rectangles = generate_rectangles(30)
print(f"Generated {len(rectangles)} rectangles.")

## Implementation: Shelf Algorithm

We implement a simple class to manage the packing.

In [ ]:
class ShelfPacker:
    def __init__(self, bin_width):
        self.bin_width = bin_width
        self.shelves = [] # List of shelves. Each shelf is {'y': start_y, 'height': h, 'items': [], 'current_x': 0}
        self.current_y = 0
        
    def pack(self, rects):
        # 1. Sort by height descending (Critical for FFDH)
        sorted_rects = sorted(rects, key=lambda r: r['h'], reverse=True)
        
        for rect in sorted_rects:
            placed = False
            
            # 2. Try to fit in existing shelves
            for shelf in self.shelves:
                if self.can_fit_in_shelf(rect, shelf):
                    self.add_to_shelf(rect, shelf)
                    placed = True
                    break
            
            # 3. If not placed, create a new shelf
            if not placed:
                self.create_new_shelf(rect)
                
        return self.shelves

    def can_fit_in_shelf(self, rect, shelf):
        # Check if there is horizontal space
        if shelf['current_x'] + rect['w'] <= self.bin_width:
            # Check if the item fits vertically in this shelf (rect height <= shelf height)
            if rect['h'] <= shelf['height']:
                return True
        return False

    def add_to_shelf(self, rect, shelf):
        rect['x'] = shelf['current_x']
        rect['y'] = shelf['y']
        shelf['items'].append(rect)
        shelf['current_x'] += rect['w']

    def create_new_shelf(self, rect):
        new_shelf = {
            'y': self.current_y,
            'height': rect['h'], # Shelf height is determined by the first (tallest) item
            'items': [],
            'current_x': 0
        }
        self.add_to_shelf(rect, new_shelf)
        self.shelves.append(new_shelf)
        self.current_y += rect['h']

# Run the packer
bin_width = 100
packer = ShelfPacker(bin_width)
packed_shelves = packer.pack(rectangles)

print(f"Packed into {len(packed_shelves)} shelves. Total Height: {packer.current_y}")

## Visualization (Matplotlib)

## Step-by-Step Visualization

To understand the algorithm better, let's visualize how it makes decisions step-by-step.

**The Logic:**
1.  **Sorting:** First, we sort all rectangles by height. This is crucial for the "Shelf" strategy because the first item in a shelf determines the shelf's height. If we put a short item first, we waste space if a taller item comes later.
2.  **Placement:** We pick the next tallest rectangle.
3.  **Check Shelves:** We look at existing shelves from bottom to top.
    *   Does it fit horizontally? (`current_x + w <= bin_width`)
    *   Does it fit vertically? (`h <= shelf_height`)
4.  **New Shelf:** If it doesn't fit anywhere, we start a new shelf on top.

In [ ]:
import copy

def plot_snapshots(snapshots, bin_width):
    n_snaps = len(snapshots)
    cols = 3
    rows = (n_snaps + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()
    
    colors = plt.cm.get_cmap('tab20').colors
    
    for idx, snap in enumerate(snapshots):
        ax = axes[idx]
        shelves = snap['shelves']
        current_rect = snap['current_rect']
        step_num = snap['step']
        action = snap['action']
        
        # Draw Bin
        total_h = max([s['y'] + s['height'] for s in shelves]) if shelves else 10
        ax.add_patch(patches.Rectangle((0, 0), bin_width, total_h + 10, linewidth=1, edgecolor='black', facecolor='#f0f0f0'))
        
        # Draw Items
        for i, shelf in enumerate(shelves):
            # Draw Shelf Line
            ax.axhline(y=shelf['y'], color='gray', linestyle=':', alpha=0.5)
            ax.text(bin_width + 2, shelf['y'], f"Shelf {i}", fontsize=8, color='gray')
            
            for j, rect in enumerate(shelf['items']):
                is_new = (rect == current_rect)
                color = 'red' if is_new else colors[(i + j) % len(colors)]
                alpha = 1.0 if is_new else 0.4
                lw = 2 if is_new else 1
                
                r = patches.Rectangle((rect['x'], rect['y']), rect['w'], rect['h'], 
                                    linewidth=lw, edgecolor='black', facecolor=color, alpha=alpha)
                ax.add_patch(r)
        
        ax.set_xlim(-5, bin_width + 15)
        ax.set_ylim(-5, total_h + 15)
        ax.set_title(f"Step {step_num}: {action}")
        ax.set_aspect('equal')
    
    # Hide empty subplots
    for i in range(n_snaps, len(axes)):
        axes[i].axis('off')
        
    plt.tight_layout()
    plt.show()

# Simulate Step-by-Step
def simulate_packing(rects, bin_width):
    # Sort
    sorted_rects = sorted(rects, key=lambda r: r['h'], reverse=True)
    
    packer = ShelfPacker(bin_width)
    snapshots = []
    
    # We'll take snapshots at specific interesting steps
    # e.g., first few, and whenever a new shelf is created
    
    for i, rect in enumerate(sorted_rects):
        # We need to deepcopy rect because the packer modifies it (adds x, y)
        rect_copy = copy.deepcopy(rect)
        
        placed = False
        action = ""
        
        # Try to fit
        for shelf_idx, shelf in enumerate(packer.shelves):
            if packer.can_fit_in_shelf(rect_copy, shelf):
                packer.add_to_shelf(rect_copy, shelf)
                placed = True
                action = f"Placed in Shelf {shelf_idx}"
                break
        
        if not placed:
            packer.create_new_shelf(rect_copy)
            action = "Created New Shelf"
            
        # Record Snapshot
        # We need deepcopy of shelves to preserve state
        if i < 6 or "New Shelf" in action: # Capture first 6 steps and any new shelf creation
             snapshots.append({
                'step': i + 1,
                'shelves': copy.deepcopy(packer.shelves),
                'current_rect': rect_copy,
                'action': action
            })
            
    return snapshots

# Generate a smaller set for clear visualization
small_rects = generate_rectangles(10, min_w=20, max_w=40, min_h=10, max_h=30)
snapshots = simulate_packing(small_rects, bin_width=100)

# Filter snapshots to not show too many (max 9)
if len(snapshots) > 9:
    snapshots = snapshots[:9]

plot_snapshots(snapshots, bin_width=100)


In [ ]:
def plot_packing(shelves, bin_width, total_height):
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Draw Bin Boundary
    ax.add_patch(patches.Rectangle((0, 0), bin_width, total_height, linewidth=2, edgecolor='black', facecolor='none'))
    
    colors = plt.cm.get_cmap('tab20').colors
    
    for i, shelf in enumerate(shelves):
        # Draw Shelf Line (optional)
        plt.axhline(y=shelf['y'], color='gray', linestyle='--', alpha=0.3)
        
        for j, rect in enumerate(shelf['items']):
            color = colors[(i + j) % len(colors)]
            # Draw Rectangle
            r = patches.Rectangle((rect['x'], rect['y']), rect['w'], rect['h'], 
                                linewidth=1, edgecolor='black', facecolor=color, alpha=0.8)
            ax.add_patch(r)
            # Add ID text
            ax.text(rect['x'] + rect['w']/2, rect['y'] + rect['h']/2, str(rect['id']), 
                    ha='center', va='center', fontsize=8, color='white')

    ax.set_xlim(-5, bin_width + 5)
    ax.set_ylim(-5, total_height + 5)
    ax.set_aspect('equal')
    plt.title(f"Shelf Packing (FFDH) - Width: {bin_width}")
    plt.show()

plot_packing(packed_shelves, bin_width, packer.current_y)

## COMPAS Representation

We can convert the packed data into COMPAS geometry objects (`Polygon` or `Frame`) for further use in a CAD environment.

In [ ]:
from compas.geometry import Polygon
from compas.colors import Color
from compas_notebook.viewer import Viewer

# Convert packed rects to COMPAS Polygons
compas_polygons = []

for shelf in packed_shelves:
    for r in shelf['items']:
        # Create 4 corners
        p1 = [r['x'], r['y'], 0]
        p2 = [r['x'] + r['w'], r['y'], 0]
        p3 = [r['x'] + r['w'], r['y'] + r['h'], 0]
        p4 = [r['x'], r['y'] + r['h'], 0]
        
        poly = Polygon([p1, p2, p3, p4])
        compas_polygons.append(poly)

# Visualize in COMPAS Viewer
viewer = Viewer()
for poly in compas_polygons:
    viewer.scene.add(poly, show_points=False, facecolor=Color.green())

# Add the bin boundary
bin_poly = Polygon([[0,0,0], [bin_width,0,0], [bin_width, packer.current_y, 0], [0, packer.current_y, 0]])
viewer.scene.add(bin_poly, show_points=False, show_faces=False, show_edges=True, linecolor=Color.red())

viewer.show()